In [1]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import numpy as np
import pandas as pd
import matplotlib as plt

from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

from rdkit import Chem
from rdkit.Chem import Descriptors

In [2]:
df = pd.read_csv("aqsoldb.csv")

Let's create a list of dictionaries, where each list item is the dictionary of chemical descriptors describing a compound.

In [3]:
# creating list of molecules in dataset using RDKit and SMILES
mol_list = []

for i in df.SMILES:
    molecule = Chem.MolFromSmiles(i)
    mol_list.append(molecule)


# creating descriptors of all molecules
complete_mol_desc = []

for molecule in mol_list:
    mol_desc = {}

    for name, function in Descriptors._descList: # Descriptors._descList provides list of all descriptors in RDKit Library
        # try-catch in case the descriptor fails to produce a value
        try:
            desc_value = function(molecule)
        
        except:
            import traceback
            traceback.print_exc()

            desc_value = None

        mol_desc[name] = desc_value
    
    complete_mol_desc.append(mol_desc)

[12:49:01] WARNING: not removing hydrogen atom without neighbors
[12:49:01] WARNING: not removing hydrogen atom without neighbors
[12:49:01] WARNING: not removing hydrogen atom without neighbors
[12:49:01] WARNING: not removing hydrogen atom without neighbors
[12:49:01] WARNING: not removing hydrogen atom without neighbors
[12:49:01] WARNING: not removing hydrogen atom without neighbors
[12:49:01] WARNING: not removing hydrogen atom without neighbors
[12:49:01] WARNING: not removing hydrogen atom without neighbors
[12:49:01] WARNING: not removing hydrogen atom without neighbors
[12:49:01] WARNING: not removing hydrogen atom without neighbors
[12:49:01] WARNING: not removing hydrogen atom without neighbors
[12:49:01] WARNING: not removing hydrogen atom without neighbors
[12:49:01] WARNING: not removing hydrogen atom without neighbors
[12:49:01] WARNING: not removing hydrogen atom without neighbors
[12:49:01] WARNING: not removing hydrogen atom without neighbors
[12:49:01] WARNING: not r

We'll now convert this list of dictionaries into a pandas dataframe, then clean/process this data.

In [4]:
df_desc = pd.DataFrame(complete_mol_desc)
df_desc = df_desc.assign(Solubility = df.Solubility) # adding column of solubility values from AqSolDB to dataframe of descriptors

inf_locations = np.where(df_desc.values >= np.finfo(np.float32).max) # locating infinite values in dataframe
for i in inf_locations[0]: # replacing infinite values with None
    for j in inf_locations[1]:
        df_desc.iat[i, j] = None

df_desc = df_desc.dropna()

In [5]:
x = df_desc.drop(['Solubility'], axis=1)
y = df_desc['Solubility']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print(f"{len(x_train)} compounds in training set.")
print(f"{len(x_test)} compounds in test set.")

7272 compounds in training set.
1819 compounds in test set.


Let's train the AdaBoost model, using Grid Search for hyperparameter tuning.

In [6]:
model = AdaBoostRegressor()

param_grid = [
    {
        'loss': ['linear'], 'estimator': [DecisionTreeRegressor(max_depth=5), DecisionTreeRegressor(max_depth=10), DecisionTreeRegressor(max_depth=20)],
        'learning_rate': [0.2, 0.4, 0.6, 0.8, 1.0], 'n_estimators': [10, 15, 20, 30, 40, 50, 75, 100]
    },
    {
        'loss': ['exponential'], 'estimator': [DecisionTreeRegressor(max_depth=5), DecisionTreeRegressor(max_depth=10), DecisionTreeRegressor(max_depth=20)],
        'learning_rate': [0.2, 0.4, 0.6, 0.8, 1.0], 'n_estimators': [10, 15, 20, 30, 40, 50, 75, 100]
     }
]

grid_search = GridSearchCV(model, param_grid, scoring=["r2", "neg_root_mean_squared_error"], refit="r2", cv=2)
grid_search = grid_search.fit(x_train, y_train)

cv_results = grid_search.cv_results_

In [17]:
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best R^2: {grid_search.best_score_}\n")

for key in cv_results:
    print(key)

average_score_time = np.mean(cv_results['mean_score_time'])
print(f"\nMean inference time is {average_score_time} seconds.")
print(cv_results['param_n_estimators'])

Best parameters: {'estimator': DecisionTreeRegressor(max_depth=20), 'learning_rate': 1.0, 'loss': 'linear', 'n_estimators': 100}
Best R^2: 0.8195831474510288

mean_fit_time
std_fit_time
mean_score_time
std_score_time
param_estimator
param_learning_rate
param_loss
param_n_estimators
params
split0_test_r2
split1_test_r2
mean_test_r2
std_test_r2
rank_test_r2
split0_test_neg_root_mean_squared_error
split1_test_neg_root_mean_squared_error
mean_test_neg_root_mean_squared_error
std_test_neg_root_mean_squared_error
rank_test_neg_root_mean_squared_error

Mean inference time is 0.29272839973370235 seconds.
[10 15 20 30 40 50 75 100 10 15 20 30 40 50 75 100 10 15 20 30 40 50 75
 100 10 15 20 30 40 50 75 100 10 15 20 30 40 50 75 100 10 15 20 30 40 50
 75 100 10 15 20 30 40 50 75 100 10 15 20 30 40 50 75 100 10 15 20 30 40
 50 75 100 10 15 20 30 40 50 75 100 10 15 20 30 40 50 75 100 10 15 20 30
 40 50 75 100 10 15 20 30 40 50 75 100 10 15 20 30 40 50 75 100 10 15 20
 30 40 50 75 100 10 15 20 30 40 

Evaluation metrics :

In [8]:
y_pred = grid_search.predict(x_test)

In [9]:
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"R^2: {r2}")
print(f"RMSE: {rmse}")

R^2: 0.8643192219416622
RMSE: 0.8676071518219576


Let's graph these results, showing how n_estimators impacts the final R^2 score.

In [ ]:
means_test = cv_results['mean_test_r2']

# means_train = cv_results['mean_train_r2'] <--- whoops, forgot to set return_train_score = True :P